# Network partitioning - synthetic data
---

## Imports
---

In [70]:
import networkx as nx
import pandas as pd
#import numpy as np
import matplotlib.pyplot as plt
import os
import math
import pickle
import osmnx as ox
import geopandas as gpd 
import momepy

%matplotlib inline

## Data
---

In [71]:
ofer_network = gpd.read_file("shp_files/StreetNetwork")

In [72]:
directory = 'shp_files/polygons'
dicty = {}

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith("shp"): 
        shp = os.path.join(directory, filename).replace("\\","/")
        gdf = gpd.read_file(shp)
        dicty[filename] = process_lines_polygons(gdf, ofer_network)

In [73]:
def process_lines_polygons(polygon_data, network_data):
    '''
    reads in:
            - shapefile of polygons
            - shapefile of network
    
    returns:
            - dataframe of attributes
            - adjacency matrix of network 
    '''
    
    gdf = polygon_data
    network = network_data
    
    network = network.set_crs('epsg:2039')
    gdf = gdf.to_crs('epsg:2039')
    gdf = gdf.set_index('id')
    
    subgraphs = {}
    for index, polygon in gdf.iterrows():
        clipped = network.clip(polygon['geometry'])
        clipped = clipped[~clipped.is_empty]    
        subgraphs[index] = clipped
    
    subgraphs_nx = {}
    for i in subgraphs:
        subgraphs_nx[i] = momepy.gdf_to_nx(subgraphs[i], approach="primal")
        
    sub_graphs_dict = {}
    for key, value in subgraphs_nx.items():
        sub_graphs_dict[key] = pd.DataFrame(nx.adjacency_matrix(subgraphs_nx[key], nodelist=sorted(subgraphs_nx[key].nodes())).todense(), 
                 index = sorted(subgraphs_nx[key].nodes()),
                columns = sorted(subgraphs_nx[key].nodes()))
    
    return(gdf, sub_graphs_dict)

In [76]:
dicty.keys()

dict_keys(['polygon1.shp', 'polygon2.shp', 'polygon_all.shp', 'polygon_merged_2.shp'])